In [7]:
import openai
import json

client = openai.OpenAI()
messages = []

In [2]:
def get_weather(city):
    return "33 degrees celcius."


FUNCTION_MAP = {
    "get_weather": get_weather,
}

In [3]:
TOOLS = [
    {
        "type": "function",
        "function": {
            "name": "get_weather",
            "description": "A function to get the weather of a city.",
            "parameters": {
                "type": "object",
                "properties": {
                    "city": {
                        "type": "string",
                        "description": "The name of the city to get the weather of. Whole words is translate to english",
                    }
                },
                "required": ["city"],
            },
        },
    }
]

In [ ]:
from openai.types.chat import ChatCompletionMessage


def process_ai_response(message: ChatCompletionMessage):
    if message.tool_calls > 0:
        messages.append(
            {
                "role": "assistant",
                "content": message.content or "",
                "tool_calls": [
                    {
                        "id": tool_call.id,
                        "type": "function",
                        "function": {
                            "name": tool_call.function,
                            "arguments": tool_call.function.arguments,
                        },
                    }
                    for tool_call in message.tool_calls
                ],
            }
        )
        ## description
        a = 123
        for tool_call in message.tool_calls:
            function_name = tool_call.function.name
            arguments = tool_call.function.arguments

            print(f"Calling functionL: {function_name} with {arguments}")

            try:
                arguments = json.loads(arguments)
            except json.JSONDecodeError:
                arguments = {}

            function_to_run = FUNCTION_MAP.get(function_name)

            result = function_to_run(**arguments)

In [ ]:
def call_ai():
    response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=messages,
            tools=TOOLS,
        )
        process_ai_response(response.choices[0].message)
    print(response)
    message = response.choices[0].message.tool_calls
    messages.append({"role": "assistant", "content": message})
    print(f"AI: {message}")

In [4]:
while True:
    message = input("LLM에게 메세지를 보내세요...")
    if message == "quit" or message == "q":
        break
    else:
        messages.append({"role": "user", "content": message})
        print(f"User: {message}")
        call_ai()

User: 서울의 날씨는 어때?
ChatCompletion(id='chatcmpl-CDCcvcJiERljVGvxPeLB3EJIaK4Uq', choices=[Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content=None, refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=[ChatCompletionMessageFunctionToolCall(id='call_XkfH5uQuDkCMQTxvWTqQyzl1', function=Function(arguments='{"city":"Seoul"}', name='get_weather'), type='function')]))], created=1757262837, model='gpt-4o-mini-2024-07-18', object='chat.completion', service_tier='default', system_fingerprint='fp_8bda4d3a2c', usage=CompletionUsage(completion_tokens=15, prompt_tokens=74, total_tokens=89, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0)))
AI: None
